# 誤差逆伝播法

単純なレイヤの実装

In [3]:
# 乗算レイヤの実装
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    # 順伝播
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    # 逆伝播
    def backward(self, dout):
        # xとyをひっくり返す
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [4]:
# 100円のりんご5個に消費税1.1を掛ける場合
apple = 100
apple_num = 5
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
# りんごの個数 * りんごの値段
apple_price = mul_apple_layer.forward(apple, apple_num)
# ↑の合計 * 消費税
price = mul_tax_layer.forward(apple_price, tax)

# 想定:220
print(price)

550.0


In [5]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
# 2.2 110 200
print(dapple, dapple_num, dtax)

5.5 110.00000000000001 500


In [6]:
# 加算レイヤの実装
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print("順伝播での出力 想定 => 715 結果 => {0}".format(price))
print("逆伝播での出力 想定 => 110 2.2 3.3 165 650 結果 => {0} {1} {2} {3} {4}".format(dapple_num, dapple, dorange, dorange_num, dtax))

順伝播での出力 想定 => 715 結果 => 715.0000000000001
逆伝播での出力 想定 => 110 2.2 3.3 165 650 結果 => 110.00000000000001 2.2 3.3000000000000003 165.0 650


In [8]:
# ReLUレイヤ
class Relu:
    def __init__(self):
        # trueとfalseを保持するインスタンス変数
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [9]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [10]:
# Sigmoidレイヤ
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [12]:
# Affineレイヤ
# データが2個ある場合
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [14]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [15]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [16]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [17]:
# Affineレイヤの実装
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(seld.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [18]:
# Softmax-with-Lossレイヤ
# ソフトマックス関数は入力された値を正規化(出力の和が1になるよう変形)して出力する
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 損失
        self.y = None # softmaxの出力
        self.t = None # 教師データ(one-hot vector)
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

①ミニバッチ:訓練データの中からランダムに一部のデータを選び出す

②勾配の算出:各重みパラメータに関する損失関数の勾配を求める

③パラメータの更新:重みパラメータを勾配方向に微小量だけ更新する

④以上を繰り返す